
# Introductory example - Job Info API

This example will show how to get information
about a job after the fact.


In [ ]:
import json
import time
import os
import flux
from flux.job import JobspecV1
import subprocess

Here we instantiate a flux handle. This will connect to the running flux instance.
If you were running this on a cluster with Flux, you'd likely already be able to
connect. If you are testing out on your own, you might need to do flux start --test-size=4



In [ ]:
handle = flux.Flux()

This is a new jobspec, or a recipe for a flux job. You'll notice we are providing a command
directly, along with tasks, nodes, and cores per task. You could also provide a script here.
If we were doing this on the command line, it would be equivalent to what is generated by:
flux submit --ntasks=4 --nodes=2 --cores-per-task=2 sleep 10



In [ ]:
jobspec = JobspecV1.from_command(
    command=["hostname"], num_tasks=1, num_nodes=1, cores_per_task=1
)

This is how we set the "current working directory" (cwd) for the job



In [ ]:
jobspec.cwd = os.getcwd()

This is how we set the job environment



In [ ]:
jobspec.environment = dict(os.environ)

Let's submit the job! We will get the job id.



In [ ]:
jobid = flux.job.submit(handle, jobspec)
time.sleep(2)

Now let's say we store that jobid somewhere how do we get info later?
We know that if we ran flux jobs -a on the command line, we'd see the job



In [ ]:
res = subprocess.getoutput('flux jobs -a')
print(res)

And if you are an expert user, you know that you can see metadata for a job
This command, without a key, will show the keys available to you



In [ ]:
res = subprocess.getoutput(f'flux job info {jobid} | true')
print(res)

And since the underlying logic here is pinging the flux KVS or key value store,
we can select one of those keys to view. For example, here is the jobspec



In [ ]:
res = subprocess.getoutput(f'flux job info {jobid} jobspec')
print(res)

This is great, but ideally we can get this metadata directly from Python.
First, here is a way to get basic jobinfo. Given we start with a string jobid,
we will first want to parse it back into a Flux JobID, and then prepare
a payload to the Job List RPC to say "give me all the attributes back"



In [ ]:
fluxjob = flux.job.JobID(jobid)
payload = {"id": fluxjob, "attrs": ["all"]}
rpc = flux.job.list.JobListIdRPC(handle, "job-list.list-id", payload)
jobinfo = rpc.get_job()
print(json.dumps(jobinfo, indent=4))

You can get less commonly used (and thus exposed) metadata like this
such as the emoji state!



In [ ]:
info = rpc.get_jobinfo()
print(info.__dict__)

But for either of the above approaches, we aren't getting anything back about our
original jobspec! That's because we need to query the KVS for that. Notice here we
have metadata like the current working directory (cwd)



In [ ]:
kvs = flux.job.job_kvs(handle, jobid)
jobspec =  kvs.get('jobspec')
print(json.dumps(jobspec))
time.sleep(2)

Finally, to watch (or stream) output, you can do the following.
Each line here is a json structure that you can further parse.
As an example, if "data" is present as a key, this usually is output



In [ ]:
for line in flux.job.event_watch(handle, jobid, "guest.output"):
    print(line)